In [ ]:
import pandas as pd
import numpy as np

#constants
num_classes = 70 # the number of breeds we want to classify
seed = 42 # makes the random numbers in numpy predictable
im_size = 299 # This size of the images
batch_size = 32

#Read the csv into a dataframe, group the breeds and 
df = pd.read_csv('../input/dog-breed-identification/labels.csv')
selected_breed_list = list(df.groupby('breed').count().sort_values(by='id', ascending=False).head(num_classes).index)
df = df[df['breed'].isin(selected_breed_list)]
#print(df)
df['filename'] = df.apply(lambda x: ('train/' + x['id'] + '.jpg'), axis=1)
#print(df['filename'])

breeds = pd.Series(df['breed'])
print("total number of breeds to classify",len(breeds.unique()))
#print(breeds)

In [ ]:
from keras.preprocessing import image

def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    path =  train_or_test + "/" + img_id + ".jpg"
    img = image.load_img(path, target_size=size)
    return image.img_to_array(img)

In [ ]:

from sklearn.preprocessing import LabelEncoder
#Declaring the Instance
label_enc = LabelEncoder()
np.random.seed(seed=seed)
rnd = np.random.random(len(df))
#This returns an Array of Boolean
train_idx = rnd < 0.9
valid_idx = rnd >= 0.9

#Here We are Encoding and Spliting the Data into Testing and Training Set.
y_train = label_enc.fit_transform(df["breed"].values)
ytr = y_train[train_idx]
yv = y_train[valid_idx]

In [ ]:
from tqdm import tqdm
#Importing the Pretrained Neural Network(For Transfer Learning)
from tensorflow.keras.applications import xception

#Creating Placeholder for Image Array(Training and Validation Images With 3 Colour Channels i.e Depth)
x_train = np.zeros((train_idx.sum(), im_size, im_size, 3), dtype='float32')
x_valid = np.zeros((valid_idx.sum(), im_size, im_size, 3), dtype='float32')
train_i = 0
valid_i = 0
for i, img_id in tqdm(enumerate(df['id'])):
    #Reading the Image using Pre-Processing Function Created Before
    img = read_img(img_id, '../input/dog-breed-identification/train', (im_size, im_size))
    #Here We are Normalizing the Image. It Takes the Numpy Array of Image and Converts into the Format that 
    #Xception Neural Network Reocgs i.e. Value Between [-1,1]
    x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
    #Block Execuites If Array Contains True(Bool)
    if train_idx[i]:
        x_train[train_i] = x
        train_i += 1
        #Adding It To the Training Set
    elif valid_idx[i]:
        x_valid[valid_i] = x
        valid_i += 1
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#Declaring the Class For Image Augmentation Process(Options are Parameters)
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

#Generating the Image From Fectures Declared
train_generator = train_datagen.flow(x_train, 
                                     ytr, 
                                     batch_size=batch_size)


valid_datagen = ImageDataGenerator()

valid_generator = valid_datagen.flow(x_valid, 
                                     yv, 
                                     batch_size=batch_size)

In [ ]:
#Declaring the Neural Network model
from keras.layers import GlobalAveragePooling2D, Dense,MaxPool2D, BatchNormalization, Dropout, Conv2D, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.models import Model, Input

# create the base pre-trained model
# We are Not Including the Weights of Last Layer(Modifying)
base_model = xception.Xception(weights='imagenet', include_top=False)
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False


# Add a global spatial average pooling layer
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer and set it to the number of breeds we want to classifiy, 
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:

import datetime
from keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 10
learning_rate = 0.001

# checkpoints
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
STAMP = "{}_dog_breed_model".format(datetime.date.today().strftime("%Y-%m-%d"))

#Saving Our Trained Model
bst_model_path = "{}.h5".format(STAMP)
model_checkpoint = ModelCheckpoint(bst_model_path,
                                   save_best_only=True,
                                   save_weights_only=False,
                                  verbose=1)



# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

hist = model.fit(train_generator,
                           steps_per_epoch=train_idx.sum() // batch_size,
                           epochs=epochs, callbacks=[early_stopping, model_checkpoint],
                           validation_data=valid_generator,
                           validation_steps=valid_idx.sum() // batch_size)

# serialize weights to HDF5
model.save(bst_model_path)
print("Saved model to disk")

In [ ]:
import matplotlib.pyplot as plt
print(hist.history.keys())
# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os


#This Function is For Pre-Processing the Image For Feeding into Neural Network
def predict_from_image(img_path):

    img = image.load_img(img_path, target_size=(299, 299))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]
    
    pred = model.predict(img_tensor)
    sorted_breeds_list = sorted(selected_breed_list)
    #print(sorted_breeds_list)
    print(np.argmax(pred))
    predicted_class = sorted_breeds_list[np.argmax(pred)]
    print(f"{predicted_class}\n{max(pred)[sorted_breeds_list.index(predicted_class)]}")
    plt.imshow(img_tensor[0])
    plt.axis('off')
    plt.show()
    
    return predicted_class, max(pred),sorted_breeds_list

In [ ]:
#Getting Image From URL
#!wget https://th.bing.com/th/id/OIP.f-lUrIFOv1OiHBwMX_KEdwHaJq?w=151&h=198&c=7&r=0&o=5&pid=1.7

#Testing This With My Own Pug Dog Image Taken In Phone
img_path = '../input/testimages/OIP.jpg'    # dog
predict_from_image(img_path)
